In [1]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-11 17:24:51.341955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746973491.366554   18910 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746973491.374189   18910 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746973491.394744   18910 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746973491.394764   18910 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746973491.394767   18910 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 7150

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="qwen1_5b-v100-bs_12_2-1epoch-webglm_ft/merged_model",
    max_seq_length=max_seq_length,
    load_in_8bit=False,
    load_in_4bit=False,
    full_finetuning=False,
    device_map="auto",
    local_files_only=True
)

==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.733 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=False,
    use_gradient_checkpointing="unsloth",
    random_state=42,
    loftq_config=None,
)

print(model.print_trainable_parameters())

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820
None


In [4]:
import json
from datasets import Dataset

def read_jsonl(file_name):
    with open(file_name, encoding="utf-8") as r:
        return [json.loads(line) for line in r]
        # return json.load(r)
    
simple_data = read_jsonl("../Semyon/data/train/synth_needle_simple_train.jsonl")
simple_val_data = read_jsonl("../Semyon/data/val/synth_needle_simple_val.jsonl")
simple_dataset = Dataset.from_list(simple_data)
simple_val_dataset = Dataset.from_list(simple_val_data)


hard_data = read_jsonl("../Semyon/data/train/synth_needle_hard_train.jsonl")
hard_val_data = read_jsonl("../Semyon/data/val/synth_needle_hard_val.jsonl")
hard_dataset = Dataset.from_list(hard_data)
hard_val_dataset = Dataset.from_list(hard_val_data)

In [ ]:
# custom_system_message = {
#     "role": "system", 
#     "content": "Ты — экспертная система Compressa RAG. Предоставляющая точные и релевантные ответы на вопросы."
# }

# # def generate_conversation(examples):
# #     conversations = []
# #     for messages in examples["messages"]:
# #         formatted_messages = [
# #             {"role": msg["role"], "content": msg["content"]}
# #             for msg in messages
# #         ]
# #         conversations.append(formatted_messages)
# #     return {"conversations": conversations}

# def generate_conversation(examples):
#     conversations = []
#     for messages in examples["messages"]:
#         formatted_messages = [custom_system_message] + [  # <- кастомный промпт
#             {"role": msg["role"], "content": msg["content"]}
#             for msg in messages
#         ]
#         conversations.append(formatted_messages)
#     return {"conversations": conversations}

# train = tokenizer.apply_chat_template(
#     dataset.map(generate_conversation, batched=True)["conversations"],
#     tokenize=False,
# )

# val = tokenizer.apply_chat_template(
#     val_dataset.map(generate_conversation, batched=True)["conversations"],
#     tokenize=False,
# )


Map:   0%|          | 0/34640 [00:00<?, ? examples/s]

Map:   0%|          | 0/1011 [00:00<?, ? examples/s]

In [5]:
simple_train = tokenizer.apply_chat_template(
    simple_dataset["messages"],
    tokenize=False,
)

simple_val = tokenizer.apply_chat_template(
    simple_val_dataset["messages"],
    tokenize=False,
)



hard_train = tokenizer.apply_chat_template(
    hard_dataset["messages"],
    tokenize=False,
)

hard_val = tokenizer.apply_chat_template(
    hard_val_dataset["messages"],
    tokenize=False,
)

In [6]:
import pandas as pd
from datasets import Dataset


simple_train_tmp = pd.Series(simple_train)
simple_val_tmp = pd.Series(simple_val)

simple_train_tmp.name = "text"
simple_val_tmp.name = "text"

simple_train_dataset = Dataset.from_pandas(pd.DataFrame(simple_train_tmp))
simple_train_dataset = simple_train_dataset.shuffle(seed = 3407)
simple_val_dataset = Dataset.from_pandas(pd.DataFrame(simple_val_tmp))




hard_train_tmp = pd.Series(hard_train)
hard_val_tmp = pd.Series(hard_val)

hard_train_tmp.name = "text"
hard_val_tmp.name = "text"

hard_train_dataset = Dataset.from_pandas(pd.DataFrame(hard_train_tmp))
hard_train_dataset = hard_train_dataset.shuffle(seed = 3407)
hard_val_dataset = Dataset.from_pandas(pd.DataFrame(hard_val_tmp))

In [ ]:
import numpy as np

tokens_count = 0
arr = []

for idx, line in enumerate(simple_train):
    l = len(tokenizer.encode(line, add_special_tokens=False))
    tokens_count += l
    arr.append(l)

arr = np.sort(np.array(arr))

print(f"tokens_avg {tokens_count / len(simple_train)}")
print(f"tokens_max {arr[-1]}")
print(f"tokens_q95 {arr[int(len(simple_train)*0.95)]}")
print(f"tokens_q99 {arr[int(len(simple_train)*0.99)]}")

tokens_avg 6727.907
tokens_max 7102
tokens_q95 6937
tokens_q99 7013


In [ ]:
tokens_count = 0
arr = []

for idx, line in enumerate(hard_train):
    l = len(tokenizer.encode(line, add_special_tokens=False))
    tokens_count += l
    arr.append(l)

arr = np.sort(np.array(arr))

print(f"tokens_avg {tokens_count / len(hard_train)}")
print(f"tokens_max {arr[-1]}")
print(f"tokens_q95 {arr[int(len(hard_train)*0.95)]}")
print(f"tokens_q99 {arr[int(len(hard_train)*0.99)]}")

tokens_avg 6518.032
tokens_max 7087
tokens_q95 6814
tokens_q99 6939


In [7]:
print(len(hard_train_dataset))

1500


In [8]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hard_train_dataset,
    eval_dataset = hard_val_dataset,
    packing = False,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 3,
        gradient_accumulation_steps = 6,
        per_device_eval_batch_size = 2,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 5e-5,
        logging_steps = 2,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to="wandb",
        output_dir="qwen1_5b-v100-synth",
        eval_steps=20,
        eval_strategy="steps",
        dataloader_num_workers=8
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/1500 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/100 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 1 | Total steps = 83
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 6
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 6 x 1) = 18
 "-____-"     Trainable parameters = 18,464,768/1,562,179,072 (1.18% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: seba-vicin (seba-vicin-oxford) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,1.294100,1.291836
40,1.271600,1.269974
60,1.264000,1.263020
80,1.262600,1.262058


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [10]:
torch.save(model.state_dict(), "qwen1_5b-v100-synth/model.pth")

In [11]:
model.save_pretrained("qwen1_5b-v100-synth/pretrain_save")

In [15]:
torch.cuda.empty_cache()